In [1]:
# importing required libraries
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import pickle 
from os import path

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

#import lightgbm as lgb
#import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

from sklearn.metrics import f1_score

import time
import datetime

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.mixed_precision import experimental as mixed_precision

# mixed - dtype 정책설정

In [4]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 3070 Laptop GPU (UUID: GPU-ee7f401a-8dfd-8593-41bd-c2179a0cf87a)


In [4]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070 Laptop GPU, compute capability 8.6
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale


In [6]:
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: float16
Variable dtype: float32


In [5]:
data = pd.read_csv('dataset_original.csv')

data.head()
new_data = data.drop(columns=['Flow_ID', 'Src_IP', 'Dst_IP', 'Timestamp'])

new_data.head()

,Src_Port,Dst_Port,Protocol,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
0,10000,10101,17,75,1,1,982,1430,982,982,...,0.0,0,0,75.0,0.000000,75,75,Anomaly,Mirai,Mirai-Ackflooding
1,2179,554,6,5310,1,2,0,0,0,0,...,0.0,0,0,2655.0,2261.327486,4254,1056,Anomaly,DoS,DoS-Synflooding
2,52727,9020,6,141,0,3,0,2806,0,0,...,0.0,0,0,70.5,0.707107,71,70,Anomaly,Scan,Scan Port OS
3,52964,9020,6,151,0,2,0,2776,0,0,...,0.0,0,0,151.0,0.000000,151,151,Anomaly,Mirai,Mirai-Hostbruteforceg
4,36763,1900,17,153,2,1,886,420,452,434,...,0.0,0,0,76.5,0.707107,77,76,Anomaly,Mirai,Mirai-Hostbruteforceg


In [6]:
# using minmax scaler for normalizing data
minmax_scale = MinMaxScaler(feature_range=(0, 1))
def normalization(df,col):
  for i in col:
    arr = df[i]
    arr = np.array(arr)
    df[i] = minmax_scale.fit_transform(arr.reshape(len(arr),1))
  return df

In [7]:
sub_cat_data = new_data.drop(columns= ['Label', 'Cat'])

In [8]:
num_col = sub_cat_data.select_dtypes(include='number').columns

In [9]:
sub_cat_data = sub_cat_data.replace([np.inf, -np.inf], np.nan)
sub_cat_data = sub_cat_data.dropna()
sub_cat_data = normalization(sub_cat_data, num_col)

In [10]:
sub_cat_data.shape

(625415, 80)

In [11]:
sub_cat_data.dropna(inplace= True)

In [12]:
sub_cat_data.shape

(625415, 80)

In [13]:
sub_cat_data

,Src_Port,Dst_Port,Protocol,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,...,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Sub_Cat
0,0.152672,0.154518,1.000000,0.000740,0.005376,0.000000,0.008940,0.001849,0.670765,0.670765,...,0.0,0.0,0.0,0.0,0.0,0.000750,0.000000,0.000750,0.000750,Mirai-Ackflooding
1,0.033267,0.008475,0.352941,0.053099,0.005376,0.001789,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.026557,0.033715,0.042551,0.010563,DoS-Synflooding
2,0.804992,0.137982,0.352941,0.001400,0.000000,0.003578,0.000000,0.003629,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000705,0.000011,0.000710,0.000700,Scan Port OS
3,0.808611,0.137982,0.352941,0.001500,0.000000,0.001789,0.000000,0.003590,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.001510,0.000000,0.001510,0.001510,Mirai-Hostbruteforceg
4,0.561267,0.029065,1.000000,0.001520,0.010753,0.000000,0.008066,0.000543,0.308743,0.296448,...,0.0,0.0,0.0,0.0,0.0,0.000765,0.000011,0.000770,0.000760,Mirai-Hostbruteforceg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625778,0.856672,0.123036,1.000000,0.002760,0.005376,0.000000,0.000164,0.000023,0.012295,0.012295,...,0.0,0.0,0.0,0.0,0.0,0.002771,0.000000,0.002771,0.002771,Mirai-UDP Flooding
625779,0.069771,0.008475,0.352941,0.016573,0.000000,0.001789,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.016584,0.000000,0.016584,0.016584,DoS-Synflooding
625780,0.805176,0.137982,0.352941,0.000760,0.005376,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000770,0.000000,0.000770,0.000770,Scan Port OS
625781,0.137710,0.761561,0.352941,0.002390,0.010753,0.000000,0.025272,0.001795,0.948087,0.948087,...,0.0,0.0,0.0,0.0,0.0,0.001200,0.000105,0.001250,0.001150,Normal


# 원핫인코딩

In [14]:
ohe=pd.get_dummies(sub_cat_data['Sub_Cat'])
print(ohe)

        DoS-Synflooding  MITM ARP Spoofing  Mirai-Ackflooding  \
0                     0                  0                  1   
1                     1                  0                  0   
2                     0                  0                  0   
3                     0                  0                  0   
4                     0                  0                  0   
...                 ...                ...                ...   
625778                0                  0                  0   
625779                1                  0                  0   
625780                0                  0                  0   
625781                0                  0                  0   
625782                0                  0                  0   

        Mirai-HTTP Flooding  Mirai-Hostbruteforceg  Mirai-UDP Flooding  \
0                         0                      0                   0   
1                         0                      0                   0 

In [15]:
X = sub_cat_data.iloc[:, :-1]
y = ohe

In [16]:
X_subset= X.iloc[600000:610000,:]
y_subset=y.iloc[600000:610000,:]

X_subset.to_csv('x_subset.csv', header=False)
y_subset.to_csv('y_subset.csv',header=False)

In [17]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=777)

In [18]:
from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation 

In [19]:
model = Sequential()
model.add(Dense(1024,input_dim=79,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(768,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(512,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(256,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(128,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(9, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              81920     
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 768)               787200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 768)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               393728    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               1

# mixed - model 구축 

In [34]:
model = Sequential()
model.add(Dense(1024,input_dim=79,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(768,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(512,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(256,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(128,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(9, activation='softmax', dtype='float32', name = 'predictions'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



In [35]:
optimizer = keras.optimizers.Adam()
optimizer = mixed_precision.LossScaleOptimizer(optimizer, loss_scale='dynamic')

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)


  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)


In [36]:
loss_object = tf.keras.losses.categorical_crossentropy

In [37]:
def train_step(x, y):
  with tf.GradientTape() as tape:
    predictions = model(x)
    loss = loss_object(y, predictions)
    scaled_loss = optimizer.get_scaled_loss(loss)
  scaled_gradients = tape.gradient(scaled_loss, model.trainable_variables)
  gradients = optimizer.get_unscaled_gradients(scaled_gradients)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss

In [38]:
@tf.function
def test_step(x):
  return model(x, training=False)

In [39]:
train_dataset = (tf.data.Dataset.from_tensor_slices((X_train, y_train))
                 .shuffle(10000).batch(256))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(256)

In [40]:
import tqdm

# 모델 TFLite 변환

In [41]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

tflite_models_dir = pathlib.Path("model0726/") # tflite 모델 저장 경로 입력 
tflite_models_dir.mkdir(exist_ok=True, parents=True)

tflite_model_file = tflite_models_dir/"model_0726.tflite" # 모델 이름 작성 
tflite_model_file.write_bytes(tflite_model) # 모델 크기 확인 

INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmp3nfg9lt8\assets


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmp3nfg9lt8\assets


ConverterError: c:\Users\user\anaconda3\envs\mixed\lib\site-packages\keras\engine\sequential.py:369:0: error: 'tf.Cast' op is neither a custom op nor a flex op
c:\Users\user\anaconda3\envs\mixed\lib\site-packages\keras\engine\base_layer.py:1037:0: note: called from
c:\Users\user\anaconda3\envs\mixed\lib\site-packages\tensorflow\lite\python\tflite_keras_util.py:194:0: note: called from
c:\Users\user\anaconda3\envs\mixed\lib\site-packages\tensorflow\python\eager\def_function.py:668:0: note: called from
c:\Users\user\anaconda3\envs\mixed\lib\site-packages\tensorflow\python\framework\func_graph.py:1007:0: note: called from
c:\Users\user\anaconda3\envs\mixed\lib\site-packages\tensorflow\python\eager\function.py:3298:0: note: called from
c:\Users\user\anaconda3\envs\mixed\lib\site-packages\tensorflow\python\eager\function.py:3463:0: note: called from
c:\Users\user\anaconda3\envs\mixed\lib\site-packages\tensorflow\python\eager\function.py:3066:0: note: called from
c:\Users\user\anaconda3\envs

In [42]:
for epoch in range(1):
  epoch_loss_avg = tf.keras.metrics.Mean()
  test_accuracy = tf.keras.metrics.CategoricalAccuracy(
      name='test_accuracy')
  for x, y in tqdm.tqdm(train_dataset):
    loss = train_step(x, y)
    epoch_loss_avg(loss)
  for x, y in tqdm.tqdm(test_dataset):
    predictions = test_step(x)
    test_accuracy.update_state(y, predictions)
  print('Epoch {}: loss={}, test accuracy={}'.format(epoch, epoch_loss_avg.result(), test_accuracy.result()))

100%|██████████| 733/733 [00:02<00:00, 343.62it/s]


Epoch 0: loss=0.7892550230026245, test accuracy=0.6926582455635071


100%|██████████| 733/733 [00:02<00:00, 359.49it/s]


Epoch 1: loss=0.6633138656616211, test accuracy=0.6869766712188721


 12%|█▏        | 210/1711 [00:03<00:22, 68.11it/s]

In [113]:
import tensorflow as tf
checkpoint_filepath = 'multiclass_best_model2.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    save_best_only=True)

In [114]:
model.fit(X_train, y_train, \
    batch_size= 256, \
        epochs=100,\
            validation_data=(X_test, y_test), \
                callbacks=[model_checkpoint_callback])

Epoch 1/100
1711/1711 [==============================] - 12s 7ms/step - loss: 0.5082 - accuracy: 0.7414 - val_loss: 0.6191 - val_accuracy: 0.7049
Epoch 2/100
1711/1711 [==============================] - 11s 7ms/step - loss: 0.5235 - accuracy: 0.7348 - val_loss: 0.5496 - val_accuracy: 0.7284
Epoch 3/100
1711/1711 [==============================] - 11s 7ms/step - loss: 0.5161 - accuracy: 0.7370 - val_loss: 0.4859 - val_accuracy: 0.7545
Epoch 4/100
1711/1711 [==============================] - 12s 7ms/step - loss: 0.5132 - accuracy: 0.7388 - val_loss: 0.5459 - val_accuracy: 0.7283
Epoch 5/100
1711/1711 [==============================] - 12s 7ms/step - loss: 0.5022 - accuracy: 0.7432 - val_loss: 0.5804 - val_accuracy: 0.7238
Epoch 6/100
1711/1711 [==============================] - 11s 7ms/step - loss: 0.5127 - accuracy: 0.7378 - val_loss: 0.5499 - val_accuracy: 0.7322
Epoch 7/100
1711/1711 [==============================] - 11s 7ms/step - loss: 0.4958 - accuracy: 0.7445 - val_loss: 0.4796 -

KeyboardInterrupt: 

In [111]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_84 (Dense)            (None, 1024)              81920     
                                                                 
 dropout_70 (Dropout)        (None, 1024)              0         
                                                                 
 dense_85 (Dense)            (None, 768)               787200    
                                                                 
 dropout_71 (Dropout)        (None, 768)               0         
                                                                 
 dense_86 (Dense)            (None, 512)               393728    
                                                                 
 dropout_72 (Dropout)        (None, 512)               0         
                                                                 
 dense_87 (Dense)            (None, 256)             

# TFLite 변환 후 Post Training Quantization 

In [135]:
import tensorflow as tf
import numpy as np
import pathlib
from tensorflow import keras 
import tqdm
model = keras.models.load_model('multiclass_best_model2.h5') # best 모델 로드 

# TFLite 모델 생성 및 양자화 

In [117]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

tflite_models_dir = pathlib.Path("model0629/") # tflite 모델 저장 경로 입력 
tflite_models_dir.mkdir(exist_ok=True, parents=True)

tflite_model_file = tflite_models_dir/"model6_0629.tflite" # 모델 이름 작성 
tflite_model_file.write_bytes(tflite_model) # 모델 크기 확인 

INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmp6nxc01bz\assets


5716300

# 동적 범위 양자화

In [118]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
tflite_model_quant_file = tflite_models_dir/"model_ptq.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)

INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmph8u2v2j2\assets


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmph8u2v2j2\assets


1439960

# Float 16 양자화

In [250]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16] # float 16 적용 

tflite_fp16_model = converter.convert()
tflite_model_fp16_file = tflite_models_dir/"model6_0623_f16.tflite" # float16 양자화 모델 이름 작성 
tflite_model_fp16_file.write_bytes(tflite_fp16_model) # 모델 크기 확인 

INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmph5jovb__\assets


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmph5jovb__\assets


2861496

# 전체 정수 양자화

In [256]:
# 정수 전용 양자화 사용 
# 입력 및 출력 텐서 양자화 
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(X_train.astype('float32')).batch(1).take(100): 
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

# Save the quantized model:
tflite_model_quant_file = tflite_models_dir/"model6_intquant_0629.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpvlra3bdr\assets


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpvlra3bdr\assets
c:\Users\user\anaconda3\envs\yolov5\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


1440768

# 전체정수 양자화 / 입력, 출력값의 형태가 정수인지 확인

In [258]:
# 입력 및 출력 텐서가 정수 형식임을 확인 
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


# 인터프리터 로드 , 텐서 할당 (동적 범위 양자화)

In [165]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter_quant.allocate_tensors()

# 인터프리터 로드, 텐서 할당 (float16)

In [251]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

interpreter_fp16 = tf.lite.Interpreter(model_path=str(tflite_model_fp16_file))
interpreter_fp16.allocate_tensors()

# 인터프리터 로드, 텐서 할당 (전체 정수)

In [257]:
interpreter_int = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter_int.allocate_tensors()

# 모델 평가 

In [259]:
# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i in tqdm.tqdm(range(len(X_test))):
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(X_test.iloc[i,:], axis=0).astype(np.uint8)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  # Compare prediction results with ground truth labels to calculate accuracy.
  accurate_count = 0
  for index in tqdm.tqdm(range(len(prediction_digits))):
    if prediction_digits[index] == y_test.iloc[index,:].argmax():
      accurate_count += 1
  accuracy = accurate_count * 1.0 / len(prediction_digits)

  return accuracy

In [253]:
print(evaluate_model(interpreter))

100%|██████████| 187625/187625 [00:16<00:00, 11325.33it/s]

0.7592271818787475


In [255]:
print(evaluate_model(interpreter_fp16)) # float16 양자화 정확도

100%|██████████| 187625/187625 [00:16<00:00, 11488.20it/s]

0.7591525649566956


In [260]:
print(evaluate_model(interpreter_int)) # integer 양자화 tfllite 모델 정확도 


100%|██████████| 187625/187625 [00:21<00:00, 8587.95it/s]

0.09564823451032645
